# pytorchで簡単にニューラルネットワークを作ろう

## ニューラルネットワークとは

- できることを見ると
  - 回帰問題（数値予測）と分類問題が解けるやつ
  - 入力と期待される出力の組（教師データ）を与えればいい感じに補間してくれるやつ
    - 損失関数は上手に設定する必要がある
- 構成するパーツを見ると
  - 人工ニューロンがいっぱい繋がったやつ

## 人工ニューロンとは（数式は覚えなくていい）

- $d$ 個の入力（=入力ベクトル）$\boldsymbol{x}=(1,x_1,\dots,x_d)^{T}$ を受け、
  重み付け和を計算し、適当な関数（活性化関数）に通して出力するやつ
  - 単に内積で表せるようにダミーの $1$ を第0次元に付け足している（cf. 同次座標系）
- ニューロンの持つパラメータは
  - 重み $\boldsymbol{w}=(w_0,w_1,\dots,w_d)^{T}$
  - 活性化関数 $f$
    - しばしば同じ層のニューロンは同じ活性化関数を使う
- 出力は $f(\boldsymbol{w}^{T}\boldsymbol{x})=f(w_0+w_1x_1+\dots+w_dx_d)$

![ニューラルネットワークは多数のニューロンが接続されている](./resources/figures/neural_network.png)
![ニューロンの入出力](./resources/figures/artificial_neuron.png)

## ニューラルネットワークの万能近似定理（普遍性定理）

3層ニューラルネットワークはいくつかの条件のもと、その重みを調節することによって出力の誤差を任意の正数まで減らすことができる

現在は活性化関数はSigmoidalな関数でなくても良いとされている

## ニューラルネットワークで分類問題を解くには

$c$ クラス分類問題を解くならば、$c$ 出力のニューラルネットワークにする

入力を与えてForward計算をしたとき、出力層において最大の値を取っているノードのクラスに分類されたとする

## 文字画像分類問題を解く

### QMNISTとは

QMNISTとは、Facebook ReserchによってMNISTを再構築したデータセット。もともとのMNISTは、NISTの手書き数字データベースのうち、Special Database 3（Census Bureau局員による）とSpecial Database 1（高校生たちによる）を抽出・前処理したものであるが、ライセンスが不明瞭であるため、（BSD-styleライセンスである）QMNISTを用いる

### ニューラルネットワークに画像を入力する（一般論として）

サイズ $A \times B$、3チャンネル（RGB）固定であるとするならば、flattenして $3AB$ 次元のベクトルとして入力すれば良い

### MNISTの場合

28x28 のグレースケール画像なので、入力は28x28x1=784次元とする